[Retour au sommaire](../index.ipynb)

# Traitement de données en tables : recherche

Les données issues d'un fichier csv ne sont pas forcément exploitables immédiatement.

Plusieurs étapes sont parfois nécessaires avant de pouvoir les utiliser.



## Type des données

Les données issues de la lecture d'un fichier CSV sont converties en chaînes de caractère (string), or certaines données peuvent avoir des types particuliers :
- entier
- flottant
- ...

Il est donc utile de convertir ces données dans le bon type (transtypage)

**Exemple en Python:**

In [ ]:
data = '123'
print(f"Le type de {data} est {type(data)}")
data = int(data) # on change le type
print(f"Le type de {data} est dorénavant {type(data)}")

## Présence de doublons

Les données sont parfois mal saisies, une recherche de doublons est possible et doit s'appuyer sur la définition exacte d'un doublon dans les données:
- est considéré comme doublon 2 lignes totalement identiques?
- est considéré comme doublon 2 lignes qui ont plusieurs champs identiques?

Les mesures à appliquer en cas de découverte de doublons doivent également être clairement définies.

## Cohérence des données

- Les données d'une colonne doivent toutes être du même type (string, integer, float...)
- Des tests complémentaires peuvent égalmement être effectués (validité d'un mail par exemple)
- Que faire en cas d'absence de données ?

## Recherche de données

L'utilité principale d'un traitement de données et de **rechercher** (filtrer) uniquement certaines valeurs parmis la multitude de données présentes.

De nombreux types de filtres existent:

### Filtre par lignes

- On peut filtrer les lignes par valeur : Afficher tous les collèges dont la ville est CHATEAULIN.
- On peut filtrer par intervales :  Afficher tous les collèges dont le code postal est compris entre 29200 et 29299 (d'où l'intérêt d'avoir bien typer les données)
- On peut combiner plusieurs filtres avec des OU ou des ET...

### Filtre par colonnes

Certaines colonnes ne sont pas forcément utiles, il est donc possible d'afficher uniquement une partie des colonnes.

## TP
Vous pouvez commencer le [TP](TP1_bib_csv.ipynb).

<table style="width:100%">
    <tr>
        <td style="text-align:left;"><a href="1_indexation_tables.ipynb">1. Indexer des données</a></td>
        <td style="text-align:right;"><a href="3_tri.ipynb">3. Trier des données ></a></td>
    </tr>
</table>

[Retour au sommaire](../index.ipynb)